In [1]:
%load_ext sql

In [2]:
%sql sqlite:///tvshows.db

#### **Views** ####

In [5]:
%%sql
Select name
FROM sqlite_master  
WHERE type='table';

 * sqlite:///tvshows.db
Done.


name
reviewers
sqlite_sequence
series
reviews


In [8]:
%sql SELECT * FROM reviews LIMIT 5;

 * sqlite:///tvshows.db
Done.


id,rating,series_id,reviewer_id
1,8,1,1
2,7.5,1,2
3,8.5,1,3
4,7.7,1,4
5,8.9,1,5


In [12]:
%%sql
SELECT title, released_year, genre, rating, first_name, last_name FROM reviews
JOIN series ON series.id = reviews.series_id
JOIN reviewers ON reviewers.id = reviews.reviewer_id
LIMIT 5;

 * sqlite:///tvshows.db
Done.


title,released_year,genre,rating,first_name,last_name
Archer,2009,Animation,8,Thomas,Stoneman
Archer,2009,Animation,7.5,Wyatt,Skaggs
Archer,2009,Animation,8.5,Kimbra,Masters
Archer,2009,Animation,7.7,Domingo,Cortes
Archer,2009,Animation,8.9,Colt,Steele


In [13]:
%%sql
CREATE VIEW review_info AS
SELECT title, released_year, genre, rating, first_name, last_name FROM reviews
JOIN series ON series.id = reviews.series_id
JOIN reviewers ON reviewers.id = reviews.reviewer_id

 * sqlite:///tvshows.db
Done.


[]

In [17]:
%%sql
SELECT name
FROM sqlite_master
WHERE type = 'view';


 * sqlite:///tvshows.db
Done.


name
review_info


In [19]:
%sql SELECT * FROM review_info LIMIT 5;

 * sqlite:///tvshows.db
Done.


title,released_year,genre,rating,first_name,last_name
Archer,2009,Animation,8,Thomas,Stoneman
Archer,2009,Animation,7.5,Wyatt,Skaggs
Archer,2009,Animation,8.5,Kimbra,Masters
Archer,2009,Animation,7.7,Domingo,Cortes
Archer,2009,Animation,8.9,Colt,Steele


In [20]:
%sql SELECT genre, AVG(rating) AS average_rating FROM review_info GROUP BY genre;

 * sqlite:///tvshows.db
Done.


genre,average_rating
Animation,7.86
Comedy,8.1625
Drama,8.04375


#### **Updateables Views** ####

- You cant DELETE from a View


Las siguientes vistas **no son actualizables** (*not updatable views*):

A) Uso de funciones de agregación (SUM, COUNT, AVG, etc.)  
B) Uso de `DISTINCT`  
C) Uso de `GROUP BY`  
D) Uso de `HAVING`  
E) Uso de `UNION` o `UNION ALL`  
F) Uso de subconsultas (subquery) en la cláusula `SELECT`


In [21]:
%%sql
CREATE VIEW display_tables AS
SELECT name
FROM sqlite_master
WHERE type = 'table';

 * sqlite:///tvshows.db
Done.


[]

In [22]:
%sql SELECT * FROM display_tables;

 * sqlite:///tvshows.db
Done.


name
reviewers
sqlite_sequence
series
reviews


#### **Alterar View** ####

```sql
CREATE VIEW ordered_series AS
SELECT * FROM series ORDER BY released_year;
 
CREATE OR REPLACE VIEW ordered_series AS
SELECT * FROM series ORDER BY released_year DESC;
 
ALTER VIEW ordered_series AS
SELECT * FROM series ORDER BY released_year;
 
DROP VIEW ordered_series;

#### **HAVING** ####

In [26]:
%sql SELECT title, AVG(rating)  FROM review_info GROUP BY title;

 * sqlite:///tvshows.db
Done.


title,AVG(rating)
Archer,8.120000000000001
Arrested Development,8.08
Bob's Burgers,7.5200000000000005
Bojack Horseman,7.94
Breaking Bad,9.36
Curb Your Enthusiasm,8.120000000000001
Fargo,9.399999999999999
Freaks and Geeks,8.6
General Hospital,5.38
Halt and Catch Fire,9.9


Resumen facilito

WHERE = filtro para datos individuales (antes de agrupar).
HAVING = filtro para datos agrupados (después de usar GROUP BY).

🚦Analogía:

WHERE = revisa niños uno por uno.
HAVING = revisa los equipos ya armados.

In [27]:
%sql SELECT title, AVG(rating)  FROM review_info GROUP BY title HAVING AVG(rating) > 8.5;

 * sqlite:///tvshows.db
Done.


title,AVG(rating)
Breaking Bad,9.36
Fargo,9.399999999999999
Freaks and Geeks,8.6
Halt and Catch Fire,9.9
Stranger Things,8.766666666666667


#### **With ROLLUP** #### 

GROUP BY agrupa normal.
ROLLUP hace agrupación + subtotales + total final automáticamente.

```sql
SELECT 
    title, AVG(rating)
FROM
    full_reviews
GROUP BY title WITH ROLLUP;

```sql
SELECT 
    title, COUNT(rating)
FROM
    full_reviews
GROUP BY title WITH ROLLUP;

```sql
SELECT 
    first_name, released_year, genre, AVG(rating)
FROM
    full_reviews
GROUP BY released_year , genre , first_name WITH ROLLUP;

#### **SQL MODES** ####

# Tabla completa de SQL Modes en MySQL (con explicación)

> Fuente: MySQL 8.4 Reference Manual – Server SQL Modes [1](https://dev.mysql.com/doc/refman/8.4/en/sql-mode.html)

## Lista completa de SQL Modes

| SQL Mode | Descripción |
|---------|-------------|
| **ALLOW_INVALID_DATES** | Permite insertar fechas inválidas (solo revisa rangos básicos). |
| **ANSI_QUOTES** | Usa `"` para identificar nombres; no se permite `"` para strings. |
| **ERROR_FOR_DIVISION_BY_ZERO** | Controla si dividir entre cero genera error o warning. (Deprecado). |
| **HIGH_NOT_PRECEDENCE** | Cambia la precedencia del operador `NOT` al estilo antiguo. |
| **IGNORE_SPACE** | Permite un espacio entre nombre de función y `(`; obliga a citar identificadores con nombres de funciones. |
| **NO_AUTO_VALUE_ON_ZERO** | Evita que `0` genere valor AUTO_INCREMENT; solo `NULL` lo genera. |
| **NO_BACKSLASH_ESCAPES** | Elimina el uso de `\` como carácter de escape. |
| **NO_DIR_IN_CREATE** | Ignora `DATA DIRECTORY` y `INDEX DIRECTORY` al crear tablas. |
| **NO_ENGINE_SUBSTITUTION** | Si un motor no existe, lanza error en vez de sustituirlo silenciosamente. |
| **NO_UNSIGNED_SUBTRACTION** | Permite resultados negativos al restar enteros UNSIGNED. |
| **NO_ZERO_DATE** | Controla si `'0000-00-00'` se permite. (Deprecado). |
| **NO_ZERO_IN_DATE** | Controla si meses o días cero están permitidos. (Deprecado). |
| **ONLY_FULL_GROUP_BY** | Obliga a que columnas no agregadas aparezcan en `GROUP BY` o dependan funcionalmente. |
| **PAD_CHAR_TO_FULL_LENGTH** | Devuelve los `CHAR` con padding completo de espacios. (Deprecado). |
| **PIPES_AS_CONCAT** | Interpreta `||` como concatenación de cadenas. |
| **REAL_AS_FLOAT** | Trata `REAL` como `FLOAT`. |
| **STRICT_ALL_TABLES** | Modo estricto para todos los motores; valores inválidos → error. |
| **STRICT_TRANS_TABLES** | Modo estricto para motores transaccionales; parcial para no transaccionales. |
| **TIME_TRUNCATE_FRACTIONAL** | Inserta valores temporales truncando fracciones en vez de redondear. |

---

## 🟣 SQL Modes combinados

| SQL Mode | Incluye | Descripción |
|----------|---------|-------------|
| **ANSI** | REAL_AS_FLOAT, PIPES_AS_CONCAT, ANSI_QUOTES, IGNORE_SPACE, ONLY_FULL_GROUP_BY | Hace que MySQL se comporte cercano al estándar ANSI SQL. |
